In [1]:
import sys
sys.path.append('../')

In [2]:
from tabnet.utils.logger import init_logger
from tabnet.estimator import TabNetRegressor
from sklearn.datasets import load_boston

In [3]:
logger_dir = 'logs'
logger_name = 'TestRegression'
level = 'INFO'

logger = init_logger(logger_dir=logger_dir, logger_name=logger_name, level=level)

In [4]:
X, y = load_boston(return_X_y=True)

print(X.shape)
print(y.shape)

(506, 13)
(506,)


In [5]:
tabnet = TabNetRegressor(
    input_dims=13, output_dims=[1], logger=logger, is_cuda=False,
    reprs_dims=4, atten_dims=4, num_steps=3, num_indep=1, num_shared=1
)

In [6]:
tabnet.build(path=None)

[2021-02-21 21:51:59,762][WARNING][TabNet] Failed to load model from None


TabNetRegressor(atten_dims=4, criterions=['mse'], input_dims=13,
                logger=<RootLogger root (INFO)>, num_indep=1, num_shared=1,
                output_dims=[1], reprs_dims=4)

In [7]:
from torch.optim import Adam
from torch.optim import lr_scheduler


training_params = {
    'batch_size': 256,
    'max_epochs': 200,
    'metrics': ['mse'],
    'optimizer': Adam,
    'optimizer_params': {'lr': 0.1},
    'schedulers': [lr_scheduler.ExponentialLR],
    'scheduler_params': {'gamma': 0.99}
}


In [8]:
tabnet.fit(X, y.reshape(-1, 1), **training_params)

[2021-02-21 21:51:59,800][INFO][TabNet] Convert to inference model.
[2021-02-21 21:51:59,801][INFO][TabNet] start training.
[2021-02-21 21:51:59,802][INFO][TabNet] ******************** epoch : 0 ********************
[2021-02-21 21:52:04,726][INFO][TabNet] -------------------- train info --------------------
[2021-02-21 21:52:04,727][INFO][TabNet] total_loss : 580.7171020507812
[2021-02-21 21:52:04,728][INFO][TabNet] task_loss : 580.7161865234375
[2021-02-21 21:52:04,728][INFO][TabNet] mask_loss : -0.9409194588661194
[2021-02-21 21:52:04,729][INFO][TabNet] mean_squared_error : 22.103702545166016
[2021-02-21 21:52:04,730][INFO][TabNet] time_cost : 0.027925
[2021-02-21 21:52:04,730][INFO][TabNet] ******************** epoch : 1 ********************
[2021-02-21 21:52:08,616][INFO][TabNet] -------------------- train info --------------------
[2021-02-21 21:52:08,617][INFO][TabNet] total_loss : 497.6362609863281
[2021-02-21 21:52:08,618][INFO][TabNet] task_loss : 497.6354064941406
[2021-02-21

KeyboardInterrupt: 

In [ ]:
importance, masks = tabnet.explain(X)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(5,15))

for i in range(3):
    axs[i].imshow(masks[i].cpu().numpy()[:20])
#     axs[i].set_xlabel('features')
    axs[i].set_ylabel('samples')
    axs[i].set_title(f"mask {i}")
    axs[i].set_yticks(range(20))
#     axs[i].set_xticks(range(30))
#     axs[i].set_xticklabels(feature_names, rotation=90)

axs[3].imshow(importance.cpu().numpy()[:20, :])
axs[3].set_xlabel('features')
axs[3].set_ylabel('samples')
axs[3].set_title('importance')
axs[3].set_yticks(range(20))
axs[3].set_xticks(range(13))
axs[3].set_xticklabels(feature_names, rotation=90)
plt.show()
axs[3].set_title('importance')

In [ ]:
getattr(tabnet._model, 'tabnet_encoder')

In [9]:
0.99 ** 500

0.006570483042414603